In [1]:
import tweepy
import pandas as pd
import csv
import time
import datetime
import matplotlib.pyplot as plt
import timeit

In [2]:
%matplotlib inline

In [3]:
# Credentials

import config

auth = tweepy.OAuthHandler(config.consumer_key, config.consumer_secret)
auth.set_access_token(config.access_token, config.access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [4]:
def mine_hashtag_tweets(q="starlink", count=150, since="2020-08-18"):
    data           =  []
    for tweet in tweepy.Cursor(api.search,
                               q=q,count=count,
                               since=since,
                               tweet_mode = 'extended',
                               monitor_rate_limit=True,
                               wait_on_rate_limit=True).items():
        mined = {'tweet_id':        tweet.id,
                 'name':            tweet.user.name,
                 'screen_name':     tweet.user.screen_name,
                 'retweet_count':   tweet.retweet_count,
                 'text':            tweet.full_text,
                 'mined_at':        datetime.datetime.now(),
                 'created_at':      tweet.created_at,
                 'favourite_count': tweet.favorite_count,
                 'hashtags':        tweet.entities['hashtags'],
                 'status_count':    tweet.user.statuses_count,
                 'location':        tweet.place,
                 'source_device':   tweet.source}   
        try:
            mined['retweet_text'] = tweet.retweeted_status.full_text
        except:
            mined['retweet_text'] = 'None'
        try:
            mined['quote_text'] = tweet.quoted_status.full_text
            mined['quote_screen_name'] = tweet.quoted_status.user.screen_name
        except:
            mined['quote_text'] = 'None'
            mined['quote_screen_name'] = 'None'
        
        last_tweet_id = tweet.id
        
        print('Grabbing tweet {0} from user {1}...'.format(tweet.id, tweet.user.name))
        
        data.append(mined)
                            
    return data 

## Grab all tweets referencing Starlink:

In [ ]:
#timeit.timeit(mine_hashtag_tweets)

In [ ]:
mined_tweets = mine_hashtag_tweets(q="starlink",count=150,since="2020-08-18")

In [ ]:
mined_tweets_df= pd.DataFrame(mined_tweets)

In [ ]:
mined_tweets_df

In [ ]:
mined_tweets_df.to_pickle("./starlink_tweets.pkl")

In [ ]:
#mined_tweets_df = pd.read_pickle('YourTweets.pkl')

In [ ]:
# how many unique tweets (not QT or RT)
unique_tweets_df = mined_tweets_df.loc[(mined_tweets_df['retweet_text'] == 'None') & (mined_tweets_df['quote_screen_name'] == 'None')]

In [ ]:
unique_tweets_df